In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
from os import listdir
import h5py
from dedalus.extras import plot_tools
import dedalus.public as d3
from dedalus.extras.plot_tools import plot_bot_2d
figkw = {'figsize':(6,4), 'dpi':100}
import logging
logger = logging.getLogger(__name__)
import matplotlib.animation as Animation
from PIL import Image
import sys
print(sys.path)

['/scratch/zz3645/2nd_week_todolist', '/ext3/miniconda3/bin', '/usr/local/sbin', '/usr/local/bin', '/usr/sbin', '/usr/bin', '/sbin', '/bin', '/ext3/miniconda3/lib/python310.zip', '/ext3/miniconda3/lib/python3.10', '/ext3/miniconda3/lib/python3.10/lib-dynload', '', '/ext3/miniconda3/lib/python3.10/site-packages']


In [2]:
# Parameters
Lx, Lz = 4, 1
Nx, Nz = 256, 64
Rayleigh = 2e6
Prandtl = 1
dealias = 3/2
stop_sim_time = 50
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

In [3]:
# Bases
coords = d3.CartesianCoordinates('x', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)


In [4]:
# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
b = dist.Field(name='b', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))
tau_p = dist.Field(name='tau_p')
tau_b1 = dist.Field(name='tau_b1', bases=xbasis)
tau_b2 = dist.Field(name='tau_b2', bases=xbasis)
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=xbasis)
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=xbasis)


In [5]:
# Substitutions
kappa = (Rayleigh * Prandtl)**(-1/2)
nu = (Rayleigh / Prandtl)**(-1/2)


In [6]:
x, z = dist.local_grids(xbasis, zbasis)
ex, ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)
grad_u = d3.grad(u) + ez*lift(tau_u1) # First-order reduction
grad_b = d3.grad(b) + ez*lift(tau_b1) # First-order reduction


In [7]:
# Problem
# First-order form: "div(f)" becomes "trace(grad_f)"
# First-order form: "lap(f)" becomes "div(grad_f)"
problem = d3.IVP([p, b, u, tau_p, tau_b1, tau_b2, tau_u1, tau_u2], namespace=locals())
problem.add_equation("trace(grad_u) + tau_p = 0")
problem.add_equation("dt(b) - kappa*div(grad_b) + lift(tau_b2) = - u@grad(b)")
problem.add_equation("dt(u) - nu*div(grad_u) + grad(p) - b*ez + lift(tau_u2) = - u@grad(u)")
problem.add_equation("b(z=0) = Lz")
problem.add_equation("u(z=0) = 0")
problem.add_equation("b(z=Lz) = 0")
problem.add_equation("u(z=Lz) = 0")
problem.add_equation("integ(p) = 0") # Pressure gauge


{'LHS': Integrate(Integrate(<Field 22517053439424>)),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.float64,
 'M': 0,
 'L': Integrate(Integrate(<Field 22517053439424>)),
 'F': <Field 22517053436928>,
 'domain': <dedalus.core.domain.Domain at 0x147aa8f8b3a0>,
 'matrix_dependence': array([ True,  True]),
 'matrix_coupling': array([False,  True])}

In [8]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time


2023-09-20 15:44:45,880 subsystems 0/1 INFO :: Building subproblem matrices 1/128 (~1%) Elapsed: 0s, Remaining: 20s, Rate: 6.3e+00/s
2023-09-20 15:44:46,391 subsystems 0/1 INFO :: Building subproblem matrices 13/128 (~10%) Elapsed: 1s, Remaining: 6s, Rate: 1.9e+01/s
2023-09-20 15:44:46,946 subsystems 0/1 INFO :: Building subproblem matrices 26/128 (~20%) Elapsed: 1s, Remaining: 5s, Rate: 2.1e+01/s
2023-09-20 15:44:47,557 subsystems 0/1 INFO :: Building subproblem matrices 39/128 (~30%) Elapsed: 2s, Remaining: 4s, Rate: 2.1e+01/s
2023-09-20 15:44:48,108 subsystems 0/1 INFO :: Building subproblem matrices 52/128 (~41%) Elapsed: 2s, Remaining: 3s, Rate: 2.2e+01/s
2023-09-20 15:44:48,659 subsystems 0/1 INFO :: Building subproblem matrices 65/128 (~51%) Elapsed: 3s, Remaining: 3s, Rate: 2.2e+01/s
2023-09-20 15:44:49,210 subsystems 0/1 INFO :: Building subproblem matrices 78/128 (~61%) Elapsed: 3s, Remaining: 2s, Rate: 2.2e+01/s
2023-09-20 15:44:49,765 subsystems 0/1 INFO :: Building subprob

In [9]:

# Initial conditions
b.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
b['g'] *= z * (Lz - z) # Damp noise at walls
b['g'] += Lz - z # Add linear background


In [10]:
# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.25, max_writes=50)
snapshots.add_task(b, name='buoyancy')
snapshots.add_task(-d3.div(d3.skew(u)), name='vorticity')


In [11]:
# CFL
CFL = d3.CFL(solver, initial_dt=max_timestep, cadence=10, safety=0.5, threshold=0.05,
             max_change=1.5, min_change=0.5, max_dt=max_timestep)
CFL.add_velocity(u)


In [12]:

# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(np.sqrt(u@u)/nu, name='Re')


In [13]:

# Main loop
startup_iter = 10
n = 0
if not os.path.exists(f'Ra_{Rayleigh:.2E}__Pr_{Prandtl}'):
    os.mkdir(f'Ra_{Rayleigh:.2E}__Pr_{Prandtl}')
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            n += 1
            max_Re = flow.max('Re')
            logger.info('Iteration=%i, Time=%e, dt=%e, max(Re)=%f' %(solver.iteration, solver.sim_time, timestep, max_Re))
            ub = b.allgather_data('g')
            plot_bot_2d(b, figkw=figkw, title=f"t={solver.sim_time}")
            plt.savefig(f'Ra_{Rayleigh:.2E}__Pr_{Prandtl}/buoyancy_{n:0>4}.png', dpi=200, bbox_inches='tight')
            matplotlib.pyplot.close()
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()


2023-09-20 15:44:55,030 __main__ 0/1 INFO :: Starting main loop
2023-09-20 15:44:56,057 __main__ 0/1 INFO :: Iteration=1, Time=1.250000e-01, dt=1.250000e-01, max(Re)=0.000000
2023-09-20 15:44:56,985 __main__ 0/1 INFO :: Iteration=11, Time=1.375000e+00, dt=1.250000e-01, max(Re)=0.121663
2023-09-20 15:44:57,778 __main__ 0/1 INFO :: Iteration=21, Time=2.625000e+00, dt=1.250000e-01, max(Re)=0.276993
2023-09-20 15:44:58,564 __main__ 0/1 INFO :: Iteration=31, Time=3.875000e+00, dt=1.250000e-01, max(Re)=0.683213
2023-09-20 15:44:59,354 __main__ 0/1 INFO :: Iteration=41, Time=5.125000e+00, dt=1.250000e-01, max(Re)=1.807417
2023-09-20 15:45:00,227 __main__ 0/1 INFO :: Iteration=51, Time=6.375000e+00, dt=1.250000e-01, max(Re)=4.956754
2023-09-20 15:45:01,003 __main__ 0/1 INFO :: Iteration=61, Time=7.625000e+00, dt=1.250000e-01, max(Re)=13.926039
2023-09-20 15:45:01,781 __main__ 0/1 INFO :: Iteration=71, Time=8.875000e+00, dt=1.250000e-01, max(Re)=39.637450
2023-09-20 15:45:02,562 __main__ 0/1 IN

In [14]:

folder_dir = "snapshots"

file_paths = [os.path.join(folder_dir, file) for file in listdir(folder_dir) if
              os.path.isfile(os.path.join(folder_dir, file)) and file.endswith('.h5')]
file_paths.sort()
print(file_paths)


['snapshots/snapshots_s1.h5', 'snapshots/snapshots_s2.h5', 'snapshots/snapshots_s3.h5', 'snapshots/snapshots_s4.h5']


In [16]:
print('yo')

folder_dir = f'Ra_2.00E+06__Pr_1'
# Get a list of PNG file paths from the directory
file_paths = [os.path.join(folder_dir, f) for f in listdir(folder_dir) if
              os.path.isfile(os.path.join(folder_dir, f)) and f.endswith('.png')]
print(file_paths)

yo
['Ra_2.00E+06__Pr_1/buoyancy_0130.png', 'Ra_2.00E+06__Pr_1/buoyancy_0267.png', 'Ra_2.00E+06__Pr_1/buoyancy_0270.png', 'Ra_2.00E+06__Pr_1/buoyancy_0209.png', 'Ra_2.00E+06__Pr_1/buoyancy_0289.png', 'Ra_2.00E+06__Pr_1/buoyancy_0206.png', 'Ra_2.00E+06__Pr_1/buoyancy_0163.png', 'Ra_2.00E+06__Pr_1/buoyancy_0234.png', 'Ra_2.00E+06__Pr_1/buoyancy_0144.png', 'Ra_2.00E+06__Pr_1/buoyancy_0186.png', 'Ra_2.00E+06__Pr_1/buoyancy_0099.png', 'Ra_2.00E+06__Pr_1/buoyancy_0261.png', 'Ra_2.00E+06__Pr_1/buoyancy_0022.png', 'Ra_2.00E+06__Pr_1/buoyancy_0050.png', 'Ra_2.00E+06__Pr_1/buoyancy_0221.png', 'Ra_2.00E+06__Pr_1/buoyancy_0223.png', 'Ra_2.00E+06__Pr_1/buoyancy_0224.png', 'Ra_2.00E+06__Pr_1/buoyancy_0353.png', 'Ra_2.00E+06__Pr_1/buoyancy_0058.png', 'Ra_2.00E+06__Pr_1/buoyancy_0026.png', 'Ra_2.00E+06__Pr_1/buoyancy_0124.png', 'Ra_2.00E+06__Pr_1/buoyancy_0081.png', 'Ra_2.00E+06__Pr_1/buoyancy_0184.png', 'Ra_2.00E+06__Pr_1/buoyancy_0062.png', 'Ra_2.00E+06__Pr_1/buoyancy_0090.png', 'Ra_2.00E+06__Pr_1/bu

In [17]:

# Sort the list of file paths
file_paths.sort()
print(file_paths)

['Ra_2.00E+06__Pr_1/buoyancy_0001.png', 'Ra_2.00E+06__Pr_1/buoyancy_0002.png', 'Ra_2.00E+06__Pr_1/buoyancy_0003.png', 'Ra_2.00E+06__Pr_1/buoyancy_0004.png', 'Ra_2.00E+06__Pr_1/buoyancy_0005.png', 'Ra_2.00E+06__Pr_1/buoyancy_0006.png', 'Ra_2.00E+06__Pr_1/buoyancy_0007.png', 'Ra_2.00E+06__Pr_1/buoyancy_0008.png', 'Ra_2.00E+06__Pr_1/buoyancy_0009.png', 'Ra_2.00E+06__Pr_1/buoyancy_0010.png', 'Ra_2.00E+06__Pr_1/buoyancy_0011.png', 'Ra_2.00E+06__Pr_1/buoyancy_0012.png', 'Ra_2.00E+06__Pr_1/buoyancy_0013.png', 'Ra_2.00E+06__Pr_1/buoyancy_0014.png', 'Ra_2.00E+06__Pr_1/buoyancy_0015.png', 'Ra_2.00E+06__Pr_1/buoyancy_0016.png', 'Ra_2.00E+06__Pr_1/buoyancy_0017.png', 'Ra_2.00E+06__Pr_1/buoyancy_0018.png', 'Ra_2.00E+06__Pr_1/buoyancy_0019.png', 'Ra_2.00E+06__Pr_1/buoyancy_0020.png', 'Ra_2.00E+06__Pr_1/buoyancy_0021.png', 'Ra_2.00E+06__Pr_1/buoyancy_0022.png', 'Ra_2.00E+06__Pr_1/buoyancy_0023.png', 'Ra_2.00E+06__Pr_1/buoyancy_0024.png', 'Ra_2.00E+06__Pr_1/buoyancy_0025.png', 'Ra_2.00E+06__Pr_1/buoya

In [ ]:
# Read the images using PIL
imgs = [Image.open(f) for f in file_paths]
fig = plt.figure(figsize=(16, 4), dpi=200)
fig.patch.set_visible(False)
plt.axis('off')
# Wrap each image in a list to create a list of sequences of artists
print('start')
imgss = [[plt.imshow(img, animated=True)] for img in imgs]
print('success?')
ani = Animation.ArtistAnimation(fig, imgss, interval=250, blit=True, repeat_delay=1000)
print('create?')
# Save the animation to a file
ani.save(f'Ra_2e6__Pr_1.gif', dpi=200)

start
